# Project Overview


In this project, we built a software test automation tool using LLM. The tool can generate unit tests, integration tests taking into account the order of integration, and regression tests. In the case of a regression test, LLM must perform a minor and a major modification.


* Unit testing: Generate all the unit testing
* Integration testing
* * Determine an integration order (integration plan)
  * Develop the corresponding unit tests
  * Develop the corresponding integration tests

* Regression testing
* * Perform minor modifications (add an attribute, remove an attribute, add a method) and see the impact on the previously implemented tests
  * Perform major modifications (change of collaboration between classes) and see the impact on previously implemented tests

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr
import anthropic
import io
import subprocess
import sys

In [3]:
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig,AutoModel,AutoProcessor
import torch

In [13]:
load_dotenv(override=True)
load_dotenv('.env.anthropic', override=True)
load_dotenv('../.env.anthropic', override=True)
load_dotenv('../.env.hugging', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
hugging_face_api_key = os.getenv('HUGGING_FACE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")
    
if hugging_face_api_key:
    print(f"Huggin Face API Key exists and begins {hugging_face_api_key[:7]}")
else:
    print("Hugging Face API Key not set") 

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Huggin Face API Key exists and begins hf_GYeH


In [18]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [19]:
openai_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
qwen = "Qwen/CodeQwen1.5-7B-Chat"
llama = "meta-llama/Llama-3.2-3B-Instruct"

In [20]:
system_prompt = "You are a software test automation tool."
system_prompt += "Your goal is to generate a variety of test tasks including generating test cases, enhancing existing test cases, and even refactoring outdated scripts."
system_prompt += "You should generate unit testing for each function that exist in the class."
system_prompt += "You should generate integration testings by takin into account integration order.  Generate the corresponding unit tests  and the corresponding integration tests"
system_prompt += "You should perform regression testing. In a case of regression testing you should perform a minor and a major modification."
system_prompt += "For the minor modification you should add an attribute, remove an attribute, add a method and see the impact on the previously implemented tests."
system_prompt += "For the major modifications you shoud change the collaboration between classes and see the impact on previously implemented tests."

In [21]:
system_prompt

'You are a software test automation tool.Your goal is to generate a variety of test tasks including generating test cases, enhancing existing test cases, and even refactoring outdated scripts.You should generate unit testing for each function that exist in the class.You should generate integration testings by takin into account integration order.  Generate the corresponding unit tests  and the corresponding integration testsYou should perform regression testing. In a case of regression testing you should perform a minor and a major modification.For the minor modification you should add an attribute, remove an attribute, add a method and see the impact on the previously implemented tests.For the major modifications you shoud change the collaboration between classes and see the impact on previously implemented tests.'

In [35]:
def user_prompt(code):
    user_prompt = "Generate a unit testing, integration testing, regression testing."
    user_prompt += "Do not add comments to explain any code."
    user_prompt = code
    return user_prompt

In [36]:
def messages(code):
    return [
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt(code)}
    ]

In [37]:
#save the code that have been generated into a text file
def save_generated_code(code):
    content = code.replace("```code","").replace("```","")
    with open("automation.txt", "w") as f:
        f.write(content)

In [45]:
# generate the test using gpt
def generate_test_gpt(code):
    stream = openai.chat.completions.create(model=openai_model,messages=messages(code),stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply
    save_generated_code(reply)
        #print(fragment, end='', flush=True)

In [46]:
# Generate the test using claude

def generate_test_claude(code):
    result = claude.messages.stream(
        model = claude_model,
        max_tokens = 2000,
        system = system_prompt,
        messages = [{"role":"user","content":user_prompt(code)}]
    )
    
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply
            #print(text, end="", flush=True)
        save_generated_code(reply)

In [47]:
def generate_test(code, model):
    if model=="GPT":
        result = generate_test_gpt(code)
    elif model=="Claude":
        result = generate_test_claude(code)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [48]:
code = """
 public class Calculator {

        public int add(int a, int b) {
            return a + b;
        }

         public double divide(int a, int b) {
            if (b == 0) {
                throw new ArithmeticException("Cannot divide by zero");
            }
            return (double) a / b;
        }
    }
"""

In [50]:
with gr.Blocks() as ui:
    with gr.Row():
        current_code = gr.Textbox(label="Code",lines=10,value=code)
        generated_code = gr.Textbox(label="Generated Test",lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT","Claude"],label="Select a model",value="GPT")
        convert = gr.Button("Generate test")
    convert.click(generate_test,inputs=[current_code,model],outputs=[generated_code])
    
ui.launch(share=True)
        

* Running on local URL:  http://127.0.0.1:7870

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
